# Nouvelle approche sur les données, traitement d'image. 

## Modules

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import soundfile as sf


from scipy import signal

import librosa as lr
import librosa.feature as feat
import librosa.display


import os 

In [2]:
path_test = '/home/ibotcazou/Bureau/Master_data_science/DATAS_M2/Apprentissage_Statistique_Panloup/Data_challenge/X__test'
path = '/home/ibotcazou/Bureau/Master_data_science/DATAS_M2/Apprentissage_Statistique_Panloup/Data_challenge/X_train'
path_Y = '/home/ibotcazou/Bureau/Master_data_science/DATAS_M2/Apprentissage_Statistique_Panloup/Data_challenge/Y_train_ofTdMHi.csv'

In [3]:
file = s.path.join(path,'01105-JAM.wav')

audio,sr = librosa.load(file)

'/home/ibotcazou/Bureau/Master_data_science/DATAS_M2/Apprentissage_Statistique_Panloup/Data_challenge/X_train/01105-JAM.wav'